In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [4]:
pbp_data = pd.read_csv("./results/base-epa.csv", encoding="utf-8")

print(f"Total Plays: {len(pbp_data)}")
pbp_data.head()

Total Plays: 859583


,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock.minutes,clock.seconds,yard_line,down,distance,yards_gained,play_type,play_text,ppa,year,play_id,home_team,coef,adjusted_yardline,margin,time_remaining,log_distance,goal_to_go,under_two,abs_diff,ScoreDiff_W,game_id,DSH,drive_point,drive_result,next_drive_point,Drive_Score_Dist_W,Total_W,Total_W_Scaled,off_full_name,off_abbr,def_full_name,def_abbr,half,new_yardline,new_down,new_distance,turnover_end,next_offense,turnover_indicator,new_time_remaining,new_log_distance,new_goal_to_go,new_under_two,end_half_game,EP_start,TD,Opp_TD,FG,Opp_FG,No_Score,EP_end,EPA,Safety,Opp_Safety,next_score,next_score_pred_prob,pred_prob,bin_pred_prob
0,322430041002,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,0,32243004101,1,29,45,80,1,10,1,Rush,Chris Burns rush for 1 yard to the UMass 21.,-0.468828,2012,322430041002,Connecticut,1,80,0,1785,2.302585,False,False,0,1.0,322430041,32243004102,0,PUNT,-7,1.0,2.0,1.0,UMass,UMass,Connecticut,UConn,1,79.0,2,9,False,UMass,0,1760.0,2.197225,False,False,False,0.699295,0.405101,0.317600,0.142792,0.114353,0.010309,-0.237787,-0.937081,0.005290,0.004555,Opp_TD,0.317600,0.317600,0.30
1,322430041003,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,0,32243004101,1,29,20,79,2,9,0,Pass Incompletion,Mike Wegzyn pass incomplete.,-0.310093,2012,322430041003,Connecticut,1,79,0,1760,2.197225,False,False,0,1.0,322430041,32243004102,0,PUNT,-7,1.0,2.0,1.0,UMass,UMass,Connecticut,UConn,1,79.0,3,9,False,UMass,0,1750.0,2.197225,False,False,False,-0.237787,0.342944,0.377006,0.129940,0.130254,0.009771,-1.298674,-1.060887,0.005439,0.004647,Opp_TD,0.377006,0.377006,0.40
2,322430041005,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,0,32243004101,1,29,10,79,3,9,5,Pass Completion,Mike Wegzyn pass complete to Marken Michel for...,0.019784,2012,322430041005,Connecticut,1,79,0,1750,2.197225,False,False,0,1.0,322430041,32243004102,0,PUNT,-7,1.0,2.0,1.0,UMass,UMass,Connecticut,UConn,1,74.0,4,4,False,UMass,0,1708.0,1.386294,False,False,False,-1.298674,0.280342,0.448131,0.104936,0.146885,0.009450,-0.222502,1.076171,0.005552,0.004703,Opp_TD,0.448131,0.448131,0.45
3,322430041006,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,0,32243004101,1,28,28,74,4,4,-2,Punt,"Jeff Strait punt for 47 yards, returned by Nic...",NaN,2012,322430041006,Connecticut,1,74,0,1708,1.386294,False,False,0,1.0,322430041,32243004102,0,PUNT,-7,1.0,2.0,1.0,UMass,UMass,Connecticut,UConn,1,24.0,1,10,True,Connecticut,1,1700.0,2.302585,False,False,False,-0.222502,0.368683,0.369151,0.077220,0.114588,0.006726,-4.141266,-3.918764,0.005036,0.058596,Opp_TD,0.369151,0.369151,0.35
4,322430041008,Connecticut,Big East,UMass,Mid-American,Connecticut,UMass,0,0,32243004102,1,28,20,25,1,10,5,Rush,Lyle McCombs rush for 5 yards to the UConn 30.,0.094545,2012,322430041008,Connecticut,0,75,0,1700,2.302585,False,False,0,1.0,322430041,32243004102,7,RUSHING TD,7,1.0,2.0,1.0,Connecticut,UConn,UMass,UMass,1,70.0,2,5,False,Connecticut,0,1666.0,1.609438,False,False,False,0.747652,0.407840,0.317276,0.147482,0.109608,0.008852,0.513041,-0.234610,0.004492,0.004449,TD,0.407840,0.407840,0.40


In [ ]:
game_data = pd.DataFrame()

# downloaded files from collegefootballdata.com
def retrieveCfbDataFile(endpoint, year):
    return pd.read_csv(f"data/{endpoint}/{year}.csv", encoding='latin-1')

for i in range(2012, 2020):
    gm = retrieveCfbDataFile('games',i)
    gm['year'] = i
    gm['winner'] = np.where(gm.home_points > gm.away_points, gm.home_team, gm.away_team)
    game_data = game_data.append(gm, sort=False)

print(f"Total Games: {len(game_data)}")

In [ ]:
pbp_data = pd.merge(pbp_data, game_data[['game_id','winner', 'home_points', 'away_points']], left_on="game_id", right_on="game_id")
pbp_data['win_indicator'] = pbp_data.offense == pbp_data.winner
pbp_data.time_remaining = np.where(pbp_data.half == 1, 1800 + pbp_data.time_remaining, pbp_data.time_remaining)
pbp_data['def_EPA'] = -1 * pbp_data.EPA
pbp_data['home_EPA'] = np.where(pbp_data.offense == pbp_data.home_team, pbp_data.EPA, -1 * pbp_data.EPA)
pbp_data['away_EPA'] = -1 * pbp_data.away_EPA
pbp_data['score_diff'] = pbp_data.offense_score - pbp_data.defense_score



